In [5]:
import json
from typing import Dict

import requests
import geopandas as gpd
import pandas as pd
from shapely.geometry import shape
from shapely import from_geojson
import numpy as np
from lonboard import Map, ScatterplotLayer

In [6]:
BASE_URL = "http://127.0.0.1:8000"
FIELDS_ENDPOINT = f"{BASE_URL}/fields"
SUMMARY_ENDPOINT = f"{BASE_URL}/summary"
AGGREGATION_ENDPOINT = f"{BASE_URL}/aggregate"

In [9]:
def fetch_admin_boundaries(iso3: str, adm: str) -> gpd.GeoDataFrame:
    """Fetch administrative boundaries from GeoBoundaries API."""
    url = f'https://www.geoboundaries.org/api/current/gbOpen/{iso3}/{adm}/'
    res = requests.get(url).json()
    return gpd.read_file(res['gjDownloadURL'])

def fetch_summary_data(feature: Dict) -> pd.DataFrame:
    """Fetch summary data for each administrative feature."""
    request_payload = {
        "aoi": feature,
        "spatial_join_method": "touches",
        "fields": ["sum_pop_2020"], 
        "geometry": "point"
    }
    response = requests.post(SUMMARY_ENDPOINT, json=request_payload)
    if response.status_code != 200:
        raise Exception(f"Failed to get summary: {response.text}")
    
    summary_data = response.json()
    if not summary_data:
        print(f"Failed to get summary for {feature['id']}")
        return pd.DataFrame()  # Return an empty DataFrame if no data
    
    df = pd.DataFrame(summary_data)
    df['adm_id'] = int(feature['id'])
    df['adm_name'] = feature['properties']['shapeName']
    df['geometry'] = df['geometry'].apply(lambda geom: shape((geom)))
    return df

In [10]:
ISO3 = 'KEN'
ADM = 'ADM1'
adm_boundaries = fetch_admin_boundaries(ISO3, ADM)
geojson_str = adm_boundaries.to_json()
adm_geojson = json.loads(geojson_str)
adm_features = adm_geojson['features']

gdfs = []
for i, feature in enumerate(adm_features):
    df = fetch_summary_data(feature)
    if not df.empty:
        gdfs.append(gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326'))
        print(feature['properties']['shapeName'], i)

# Concatenate all GeoDataFrames into a single GeoDataFrame
gdf = pd.concat(gdfs, ignore_index=True)

# Display the GeoDataFrame structure
gdf.head()

Turkana 0
Marsabit 1
Mandera 2
Wajir 3
West Pokot 4
Samburu 5
Isiolo 6
Baringo 7
Elgeyo-Marakwet 8
Trans Nzoia 9
Bungoma 10
Garissa 11
Uasin Gishu 12
Kakamega 13
Laikipia 14
Busia 15
Meru 16
Nandi 17
Siaya 18
Nakuru 19
Vihiga 20
Nyandarua 21
Tharaka 22
Kericho 23
Kisumu 24
Nyeri 25
Tana River 26
Kitui 27
Kirinyaga 28
Embu 29
Homa Bay 30
Bomet 31
Nyamira 32
Narok 33
Kisii 34
Murang'a 35
Migori 36
Kiambu 37
Machakos 38
Kajiado 39
Nairobi 40
Makueni 41
Lamu 42
Kilifi 43
Taita Taveta 44
Kwale 45
Mombasa 46


,hex_id,geometry,sum_pop_2020,adm_id,adm_name
0,866a4a407ffffff,POINT (35.49277 2.72376),554.655021,0,Turkana
1,866a4a417ffffff,POINT (35.10158 4.70202),207.977046,0,Turkana
2,866a4a41fffffff,POINT (35.05497 4.66712),637.218817,0,Turkana
3,866a4a427ffffff,POINT (36.38063 2.48258),948.876730,0,Turkana
4,866a4a437ffffff,POINT (35.12864 4.85068),112.805780,0,Turkana


In [11]:
# Define custom breaks and corresponding RGBA colors for visualization
breaks = [0, 1, 1000, 10000, 50000, 100000, 200000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 255],  # Light gray for 0
    [255, 255, 0, 255],    # Yellow for 1-1000
    [255, 165, 0, 255],    # Orange for 1000-10000
    [255, 0, 0, 255],      # Red for 10000-50000
    [128, 0, 128, 255],    # Purple for 50000-100000
    [0, 0, 255, 255],      # Blue for 100000-200000
    [0, 0, 139, 255],      # Dark blue for 200000+
])

def assign_color(value: float) -> list:
    """Assign colors based on population value."""
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i].tolist()  # Convert to list
    return colors[-1].tolist()  # In case value exceeds all breaks

In [12]:
# Map sum_pop_2020 values to colors using the custom function
gdf['color'] = gdf["sum_pop_2020"].apply(assign_color)

# Flatten the color list into a 2D array for ScatterplotLayer
color_list = np.array(gdf['color'].tolist(), dtype=np.uint8)

# Create the scatterplot layer with the assigned colors
layer = ScatterplotLayer.from_geopandas(gdf, get_radius=2000, get_fill_color=color_list)

# Create the map with the scatterplot layer
m = Map(layer)
m

Map(layers=[ScatterplotLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x15dae2c80>
[
  [
    2…

In [13]:
adm_features

[{'id': '0',
  'type': 'Feature',
  'properties': {'shapeName': 'Turkana',
   'shapeISO': 'KE-43',
   'shapeID': '32016919B72266624462344',
   'shapeGroup': 'KEN',
   'shapeType': 'ADM1'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[36.05060958836185, 4.456217766236989],
     [35.94394683797623, 4.548029422619834],
     [35.93885421734831, 4.584362507224],
     [35.949348448572096, 4.628556251969485],
     [35.81214523321597, 4.782410621852762],
     [35.81184768712109, 5.092334270410277],
     [35.81184387219702, 5.095732688812575],
     [35.821193695432726, 5.106872558621717],
     [35.82409668001782, 5.109823227060303],
     [35.825977325193435, 5.111377239368153],
     [35.82680892918876, 5.112345695897204],
     [35.82710647618296, 5.113322734656549],
     [35.827430724948044, 5.115200996628744],
     [35.82780837995284, 5.116295337559848],
     [35.82852172759573, 5.117104529550431],
     [35.830711365437196, 5.118960380417263],
     [35.83223724335414, 5.1204833975907

In [16]:
def fetch_aggregated_population(row):
    request_payload = {
        "aoi": {
            "type": "Feature",
            "geometry": row.geometry,
            "properties": {}
        },
        "spatial_join_method": "touches",
        "fields": ["sum_pop_2020"],
        "aggregation_type": "sum"
    }

    response = requests.post(AGGREGATION_ENDPOINT, json=request_payload)

    if response.status_code == 200:
        result = response.json()
        return result.get("sum_pop_2020", 0)
    else:
        return 0


In [17]:
adm_gdf = gpd.GeoDataFrame.from_features(adm_features)
adm_gdf['sum_pop_2020'] = adm_gdf.apply(fetch_aggregated_population, axis=1)
adm_gdf.head()

TypeError: Object of type Point is not JSON serializable

In [16]:
breaks = [0, 10_000, 100_000, 1_000_000, 5_000_000, 10_000_000, 20_000_000, gdf["sum_pop_2020"].max()]
colors = np.array([
    [211, 211, 211, 125],  # Light gray with transparency for 0 - 10,000
    [255, 255, 0, 125],    # Yellow with transparency for 10,000 - 100,000
    [255, 165, 0, 125],    # Orange with transparency for 100,000 - 1,000,000
    [255, 0, 0, 125],      # Red with transparency for 1,000,000 - 5,000,000
    [128, 0, 128, 125],    # Purple with transparency for 5,000,000 - 10,000,000
    [0, 0, 255, 125],      # Blue with transparency for 10,000,000 - 20,000,000
    [0, 0, 139, 125],      # Dark blue with transparency for 20,000,000 - max
])

def assign_color(value: float) -> list:
    """Assign colors based on population value, including transparency."""
    for i in range(len(breaks) - 1):
        if breaks[i] <= value < breaks[i + 1]:
            return colors[i].tolist()  # Convert to list
    return colors[-1].tolist()

In [17]:
from lonboard import PolygonLayer
import numpy as np

# Assign colors to each row in gdf based on sum_pop_2020
adm_gdf['color'] = adm_gdf["sum_pop_2020"].apply(assign_color)

# Convert the list of colors into a 2D numpy array of uint8
color_array = np.array(gdf['color'].tolist(), dtype=np.uint8)

# Create the polygon layer using PolygonLayer, referencing the color_array
layer = PolygonLayer.from_geopandas(
    adm_gdf,
    get_fill_color=color_array,  # Pass the 2D numpy array for the colors
    get_line_color=[0, 0, 0, 255],  # Optional: Black outline
    line_width_min_pixels=1
)

# Create the map with the polygon layer
m = Map(layer)
m

/opt/anaconda3/envs/spacestats/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:23: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(layers=[PolygonLayer(get_fill_color=<pyarrow.lib.FixedSizeListArray object at 0x175bdc040>
[
  [
    255,
…